In [56]:
%matplotlib inline
import ipywidgets as widgets
import matplotlib.pyplot as plt

from functools import partial

import pennylane as qml
import pennylane.numpy as np
from pennylane.fourier import coefficients
from pennylane.fourier.visualize import bar

# Noise

Based on [Pennylane: How to simulate noise with PennyLane](https://pennylane.ai/blog/2021/05/how-to-simulate-noise-with-pennylane/).

Noise in Quantum Circuits can be modeled using Kraus operators acting on states described by density matrices as
$$ 
\Phi(\rho)=\sum_e K_e \rho K_e^{\dagger}, \quad \sum_e K_e^{\dagger} K_e=I
$$
where each $K_e$ is associated with a particular outcome which may occur when applying the channel.

## Random BitFlip

$$
\begin{gathered}
K_0=\sqrt{1-p}\left[\begin{array}{ll}
1 & 0 \\
0 & 1
\end{array}\right] \\
K_1=\sqrt{p}\left[\begin{array}{ll}
0 & 1 \\
1 & 0
\end{array}\right]
\end{gathered}
$$

where $p \in[0,1]$ is the probability of a bit flip (Pauli $X$ error).


## Random PhaseFlip

$$
\begin{gathered}
K_0=\sqrt{1-p}\left[\begin{array}{ll}
1 & 0 \\
0 & 1
\end{array}\right] \\
K_1=\sqrt{p}\left[\begin{array}{cc}
1 & 0 \\
0 & -1
\end{array}\right]
\end{gathered}
$$

where $p \in[0,1]$ is the probability of a phase flip (Pauli $Z$ ) error.

## Amplitude Damping

$$
\begin{gathered}
K_0=\left[\begin{array}{cc}
1 & 0 \\
0 & \sqrt{1-\gamma}
\end{array}\right] \\
K_1=\left[\begin{array}{cc}
0 & \sqrt{\gamma} \\
0 & 0
\end{array}\right]
\end{gathered}
$$

where $\gamma \in[0,1]$ is the amplitude damping probability.

## Phase Damping

$$
\begin{gathered}
K_0=\left[\begin{array}{cc}
1 & 0 \\
0 & \sqrt{1-\gamma}
\end{array}\right] \\
K_1=\left[\begin{array}{cc}
0 & 0 \\
0 & \sqrt{\gamma}
\end{array}\right]
\end{gathered}
$$

where $\gamma \in[0,1]$ is the phase damping probability.


## Depolarization Channel

$$
\begin{aligned}
K_0 & =\sqrt{1-p}\left[\begin{array}{ll}
1 & 0 \\
0 & 1
\end{array}\right] \\
K_1 & =\sqrt{p / 3}\left[\begin{array}{ll}
0 & 1 \\
1 & 0
\end{array}\right] \\
K_2 & =\sqrt{p / 3}\left[\begin{array}{cc}
0 & -i \\
i & 0
\end{array}\right] \\
K_3 & =\sqrt{p / 3}\left[\begin{array}{cc}
1 & 0 \\
0 & -1
\end{array}\right]
\end{aligned}
$$

where $p \in[0,1]$ is the depolarization probability and is equally divided in the application of all Pauli operations.

In [57]:
rng = np.random.default_rng(100)

n_qubits = 2
n_layers = 4

dev = qml.device("default.mixed", wires=n_qubits)

weights = 2*np.pi * rng.random(size=(n_layers,n_qubits*3-1))
x = 1.0

In [58]:
def pqc(w:np.ndarray):
    """
    Creates a Circuit19 ansatz.

    Length of flattened vector must be n_qubits*3-1
    because for >1 qubits there are three gates

    Args:
        w (np.ndarray): weight vector of size n_layers*(n_qubits*3-1)
    """

    w_idx = 0
    for q in range(n_qubits):
        qml.RY(w[w_idx], wires=q)
        w_idx += 1
        qml.RZ(w[w_idx], wires=q)
        w_idx += 1

        if q > 0:
            qml.CRX(w[w_idx], wires=[q, (q + 1) % n_qubits])
            w_idx += 1

def iec(x:np.ndarray):
    """
    Creates an AngleEncoding using RY gates

    Args:
        x (np.ndarray): length of vector must be 1
    """

    for q in range(n_qubits):
        qml.RY(x, wires=q)

@qml.qnode(dev)
def circuit(w:np.ndarray, x:np.ndarray, bf=0.0, pf=0.0, ad=0.0, pd=0.0, dp=0.0):
    """
    Creates a circuit with noise.
    This involves, Amplitude Damping, Phase Damping and Depolarization.
    The Circuit consists of a PQC and IEC in each layer. 

    Args:
        w (np.ndarray): weight vector of size n_layers*(n_qubits*3-1)
        x (np.ndarray): input vector of size 1
        ad (float, optional): Amplitude Damping. Defaults to 0.0.
        pd (float, optional): Phase Damping. Defaults to 0.0.
        dp (float, optional): Depolarization. Defaults to 0.0.

    Returns:
        _type_: _description_
    """
    for l in range(n_layers):
        pqc(w[l])
        iec(x)
        
        for q in range(n_qubits):
            qml.BitFlip(bf, wires=q)
            qml.PhaseFlip(pf, wires=q)
            qml.AmplitudeDamping(ad, wires=q)
            qml.PhaseDamping(pd, wires=q)
            qml.DepolarizingChannel(dp, wires=q)

    return qml.expval(qml.PauliZ(0))

In [59]:
n_samples = n_layers*10
x_data = np.linspace(0, 2*np.pi, n_samples)

colour_dict={"real" : "red", "imag" : "blue"}

In [67]:
@widgets.interact()
def plot(BitFlip=0.0, PhaseFlip=0.0, AmpDamp=0.0, PhaseDamp=0.0, Depol=0.0):
    fig, ax = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(15, 6))
    coeffs = coefficients(partial(circuit, weights, bf=BitFlip, pf=PhaseFlip, ad=AmpDamp, pd=PhaseDamp, dp=Depol), 1, n_qubits*n_layers)

    bar(coeffs, 1, ax, colour_dict=colour_dict)

    fig, ax = plt.subplots(1,1, figsize=(15, 3))
    
    y = circuit(weights, x_data, bf=BitFlip, pf=PhaseFlip, ad=AmpDamp, pd=PhaseDamp, dp=Depol)
    ax.plot(x_data, y, color="green", label="Circuit")
    ax.set_xlabel("X Domain")
    ax.set_ylabel("Expectation Value")
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

interactive(children=(FloatSlider(value=0.0, description='BitFlip', max=1.0), FloatSlider(value=0.0, descripti…